# PAE logp sampling

In [1]:

#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
#tf.disable_eager_execution()
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rcParams
import sys
import pickle
from functools import partial
import time
from tqdm import tqdm

plt.rcParams.update({'font.family' : 'lmodern', 'font.size': 16,                                                                                                                                                    
                     'axes.labelsize': 16, 'legend.fontsize': 12, 
                     'xtick.labelsize': 16, 'ytick.labelsize': 16, 'axes.titlesize': 16,
                     'axes.linewidth': 1.5})

In [2]:
import scipy

In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_hub as hub
tfd = tfp.distributions
tfb = tfp.bijectors
tf.__version__
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
print(tf.__version__, tfp.__version__, hub.__version__)

2.2.0 0.10.0 0.8.0


### Loading the trained modules and evaluating logp in tensorflow

In [5]:
import pae.create_datasets as crd
import pae.load_data as ld
load_funcs=dict(mnist=ld.load_mnist, fmnist=ld.load_fmnist)

In [6]:

PARAMS_PATH = '/global/cscratch1/sd/vboehm/PAE_samples/FMNIST/latent_dim64/modules/'

param_file  = 'params_fmnist_-1_64_infoGAN_AE_v2rot_full_sigma'
params      = pickle.load(open(os.path.join(PARAMS_PATH,param_file+'.pkl'),'rb'))

In [7]:
# params['module_dir']='../../modules/mnist/class-1/latent_size10/net_type_vae10/loss_AE/test_full_sigma'
# params['data_dir']= '../../data/'

In [8]:
load_func                                          = partial(load_funcs[params['data_set']])
x_train, y_train, x_valid, y_valid, x_test, y_test = load_func(params['data_dir'],flatten=False)

if np.all(x_test)==None:
    x_test=x_valid

x_train = (x_train/256.-0.5).astype(np.float32)
x_test  = (x_test/256.-0.5).astype(np.float32)
x_valid = (x_valid/256.-0.5).astype(np.float32)

In [9]:
load_func                                          = partial(load_funcs['mnist'])
x_train_ood, y_train, x_valid_ood, y_valid, x_test_ood, y_test = load_func(params['data_dir'],flatten=False)


x_train_ood = (x_train_ood/256.-0.5).astype(np.float32)
x_test_ood  = (x_test_ood/256.-0.5).astype(np.float32)
x_valid_ood = (x_valid_ood/256.-0.5).astype(np.float32)

In [30]:
params['module_dir']='/global/cscratch1/sd/vboehm/PAE_samples/FMNIST/latent_dim64/modules/'

'../../modules/fmnist/class-1/latent_size64/net_type_infoGAN/loss_AE/v2rot_full_sigma'

In [10]:
generator_path   = os.path.join(params['module_dir'],'decoder')
encoder_path     = os.path.join(params['module_dir'],'encoder')
nvp_path         = os.path.join(params['module_dir'],'hybrid1_nepoch100')

In [11]:
def get_likelihood(decoder,sigma):
    sigma = tf.reshape(sigma,(params['batch_size'],-1))
    def likelihood(z):
        mean  = decoder({'z':z})['x']
        mean  = tf.reshape(mean,(params['batch_size'],-1))
        
        LL = tfd.MultivariateNormalDiag(loc=mean,scale_diag=sigma)
        return tfd.Independent(LL)

    return likelihood

def get_prior(latent_size):
    return tfd.MultivariateNormalDiag(tf.zeros(latent_size), scale_identity_multiplier=1.0)

In [13]:
#z_fill        = tf.Variable(tf.zeros((params['batch_size'],params['latent_size']),tf.float32),trainable=False)

sigma         = params['full_sigma']
sigma         = sigma.astype(np.float32)
print(sigma.shape)
encoder       = hub.KerasLayer(encoder_path,trainable=False, signature_outputs_as_dict=True)
decoder       = hub.KerasLayer(generator_path, trainable=False, signature_outputs_as_dict=True)
nvp_funcs     = hub.KerasLayer(nvp_path, trainable=False, signature_outputs_as_dict=True)

likelihood    = get_likelihood(decoder,np.repeat(np.expand_dims(sigma,0),params['batch_size'],axis=0))
prior         = get_prior(params['latent_size'])

(28, 28, 1)


In [16]:
def get_encoded(x):
    mu, sigma        = tf.split(encoder({'x':x})['z'], 2, axis=-1)
    return mu

def get_decoded(z):
    return decoder({'z':z})['x']

def likelihood_eval(z,x,likelihood):
    likelihood    = likelihood(z).log_prob(x)
    return likelihood

def NF_prior_eval(z,nvp_funcs=nvp_funcs):
    prior         = nvp_funcs({'z_sample':z,'sample_size':1, 'u_sample':np.zeros((1,params['latent_size']))})['log_prob']
    return prior

def posterior_eval(z,x,likelihood, nvp_funcs):
    x            = tf.reshape(x,(params['batch_size'],-1))
    likelihood   = likelihood_eval(z,x,likelihood)
    NF_prior     = prior_eval(z, nvp_funcs)
    logprob      = likelihood+NF_prior
    return logprob

def Jacobian_eval(z,nvp_funcs):
    NF_prior = NF_prior_eval(z,nvp_funcs=nvp_funcs)
    J        = NF_prior-prior.log_prob(z)
    return J

In [19]:
samples = np.load(os.path.join('/global/cscratch1/sd/vboehm/PAE_samples/FMNIST/latent_dim64/samples','HMC_MNIST_latent_dim64_burnin50_leapfrog3_2.npy'),allow_pickle=True)

In [29]:
posteriors = np.zeros(samples.shape[:-1])

num_chunks = len(posteriors)//params['batch_size']

for ii in range(num_chunks):
    for jj in range(25):
        z  = samples[ii*params['batch_size']:(ii+1)*params['batch_size'],jj]
        post = posterior_eval(z,x_test[ii*params['batch_size']:(ii+1)*params['batch_size']],likelihood, nvp_funcs)
        posteriors[ii*params['batch_size']:(ii+1)*params['batch_size'],jj]=post

In [24]:
posteriors

(3200, 25, 64)